In [0]:
%sql
describe globalretail_bronze.bronze_transactions;

In [0]:
spark.sql("USE globalretail_silver")
spark.sql("""
CREATE TABLE IF NOT EXISTS silver_transactions (
    transaction_id STRING,
    customer_id STRING,
    product_id STRING,
    quantity INT,
    total_amount DOUBLE,
    transaction_date DATE,
    payment_method STRING,
    store_type STRING,
    order_status STRING,
    last_updated TIMESTAMP
)
USING DELTA
""")

In [0]:
%sql
select table_name 
from workspace.information_schema.tables 
where table_schema = 'globalretail_silver'

In [0]:
%sql
describe globalretail_silver.silver_transactions;

In [0]:
# Get the last processed timestamp from silver layer
last_processed_df = spark.sql("SELECT MAX(last_updated) as last_processed FROM silver_transactions")
last_processed_timestamp = last_processed_df.collect()[0]['last_processed']

if last_processed_timestamp is None:
    last_processed_timestamp = "1900-01-01T00:00:00.000+00:00"

In [0]:
# Create a temporary view of incremental bronze data
spark.sql(f"""
CREATE OR REPLACE TEMPORARY VIEW bronze_incremental_transactions AS
SELECT *
FROM globalretail_bronze.bronze_transactions WHERE ingestion_timestamp > '{last_processed_timestamp}'

""")

In [0]:
%sql
select *
from bronze_incremental_transactions
limit 10;